In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt
import re

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [ ]:
lambda_ranges =[np.array([3.5,3.2])]
# lambda_ranges =[np.array([3.5,3.2]),np.array([5.8,5.5]),np.array([6.8,6.5])]

In [ ]:
n_eff = np.sqrt((1-0.2237)+0.2237*(3.3)**2)
# n_eff=1

In [ ]:
folder_path = rf"../Structures"
project_name = "20251010_Beam_Spreading_12x"
postprocess_results = []
runtimes_ps = [8e-12,25e-12]
min_steps_per_lambda = 14
cuts=[1.5]
h5_bg = None
ref = True
for l,lambdas in enumerate(lambda_ranges): 
    for dirpath, dirnames, filenames in os.walk(folder_path):
        try:
            for filename in filenames:
                if filename.endswith(".h5") and filename=="n_3.30_ff_0.2237.h5":
                    ff = float(re.search(r'ff_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    n_value = float(re.search(r'n_([+-]?\d+(?:\.\d+)?)', filename).group(1))
                    for cut in cuts:
                        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
                            continue 
                        if os.path.isfile(os.path.join(dirpath, filename)):
                            file=os.path.join(dirpath, filename)
                            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                                            ,direction="z", lambda_range=lambdas,
                                                            box_size=14.3,runtime_ps=runtimes_ps[l],min_steps_per_lambda=min_steps_per_lambda,
                                                           scaling=1,shuoff_condtion=1e-20, verbose=True, multiplication_factor=12, multiplicate_size=True,
                                                           monitors=["time_monitor"], flux_monitor_position=18,cell_size_manual=45,
                                                           freqs=100, width=0.4,sim_bg=n_eff**2, boundaries="absorbers",
                                                           cut_condition=cut, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                                                           )
                            
                            
                            f0 = td.C_0/(np.sum(lambdas)/2)
                            source_def = td.PlaneWave(
                                source_time = td.GaussianPulse(
                                    freq0=f0,
                                    fwidth=f0/20,
                                    offset=10,
                                    phase=0
                                ),
                                size= (2.5, 
                                      2.5, 
                                      0) 
                                      ,
                                center=( 0, 
                                         0, 
                                        (-structure_1.t_slab_z/2 -1)),
                                direction='+',
                                
                                pol_angle=structure_1.pol_angle,
                                name='planewave',
                                )
                            
                            time_monitorField_transverse = td.FieldTimeMonitor(
                                    center=[0,0,0],
                                    size=[
                                            structure_1.t_slab_y,
                                            0,
                                             structure_1.t_slab_z+structure_1.spacing
                                        ],
                                    start=0,
                                    interval_space = (2,2,2),
                                    stop=structure_1.t_stop,
                                    fields=["Ex", "Ey", "Ez"],
                                    interval=300,
                                    name="time_monitorField_transverse", 
                                )
                            
                            time_monitorField_exit = td.FieldTimeMonitor(
                                    center=[0,0,structure_1.t_slab_z/2],
                                    size=[
                                            structure_1.t_slab_x,
                                            structure_1.t_slab_y,
                                            0
                                        ],
                                    start=0,
                                    interval=150,
                                    interval_space = (2,2,2),
                                    stop=structure_1.t_stop,
                                    fields=["Ex", "Ey", "Ez"],
                                    name="time_monitorField_exit",
                                )
                            
                            sim = structure_1.sim 
                            sim=sim.copy(update={"sources":[source_def],"monitors":[time_monitorField_exit]})
                            # sim.plot_3d()
                            # raise Exception
                           

                            folder_desc = rf"H:\phd stuff\tidy3d\data\{project_name}\n_{n_value:.2f}"
                            os.makedirs(folder_desc, exist_ok=True)
                            sim_name=rf"LSU_{Path(filename).stem}_size_{cut}_lambda_{lambdas[1]}_{lambdas[0]}_sim_bg_{n_eff**2:.2f}"
                            if os.path.exists(os.path.join(folder_desc, sim_name+".txt")):
                                print("Exist!")
                            else:
                                id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
                                ids = '\n' + id
                                with open(os.path.join(folder_desc, sim_name+".txt"), "w") as file:
                                    # Write the string to the file
                                    file.write(ids)
                                web.start(task_id = id)
                                web.monitor(id)

        except Exception as e:
            print(f"Error processing {dirpath}: {e}")

        
    

